In [1]:
import numpy as np
from scipy.optimize import curve_fit

In [2]:
print('hello')

hello


In [ ]:
def findQubits():
    '''
    Scans a frequency window, finds the resonances
    Inputs:
        fmin: minimum frequency
        fmax: maximum frequency
        n_qubits: estimated number of qubits in the window
    Returns:
        f_qubits: np array of qubit frequencies
    '''
    